# Mesh to Mesh Registration Example

ITK natively supports image-to-image registration, which is a common operation for medical images with symmetry. Another common method of storing 3D volumetric data is to represent volume surfaces with meshes. In this example we seek to register two meshes using various ITK metrics and optimization techniques.

Registration classes are defined in the Python `hasi` submodule and built on top of the ITK Python wrapping. The `MeanSquaresRegistrar` and `DiffeoRegistrar` classes apply registration techniques to images derived from mesh inputs, while the `PointSetEntropyRegistrar` aims to register meshes via point set entropy metrics. Mesh registration is carried out with each class in this notebook on sample bone femur mesh data downloaded to the `examples/Data` folder.

This notebook requires the following modules, which can be either acquired via `pip` or built alongside the ITK `master` branch:
- [ITK](https://github.com/InsightSoftwareConsortium/ITK/)
- [ITKBoneEnhancement](https://github.com/InsightSoftwareConsortium/ITKBoneEnhancement)
- [ITKMeshToPolyData](https://github.com/InsightSoftwareConsortium/ITKMeshToPolyData)
- [ITKWidgets](https://github.com/InsightSoftwareConsortium/itkwidgets)

In [1]:
# Update sys.path to reference src/ modules
import os
import sys
import copy
import importlib
from urllib.request import urlretrieve

import itk
from itkwidgets import view, checkerboard, compare
from ipywidgets import FloatProgress, Label, HBox, VBox, FloatText, ColorPicker, Button
PATTERN_COUNT = 5

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
os.makedirs('Input', exist_ok=True)
os.makedirs('Output', exist_ok=True)

In [3]:
MESH_TO_USE = '901-R'
TARGET_MESH_FILE = f'Input/meshes/{MESH_TO_USE}-mesh.vtk'#f'Input/{MESH_TO_USE}-mesh.vtk'
TEMPLATE_MESH_FILE = f'Input/906-R-atlas.obj'

MEANSQUARES_OUTPUT_FILE = f'Output/{MESH_TO_USE}-meansquares-registered.obj'
DIFFEO_OUTPUT_FILE = f'Output/{MESH_TO_USE}-diffeo-registered.obj'
POINTSET_OUTPUT_FILE = f'Output/{MESH_TO_USE}-pointset-registered.obj'
POINTSET_RESAMPLED_OUTPUT_FILE = f'Output/{MESH_TO_USE}-pointset-resampled.obj'

In [4]:
# Download meshes
if not os.path.exists(TARGET_MESH_FILE):
    url = 'https://data.kitware.com/api/v1/file/5f9daaba50a41e3d1924dae9/download'
    urlretrieve(url, TARGET_MESH_FILE)
if not os.path.exists(TEMPLATE_MESH_FILE):
    url = 'https://data.kitware.com/api/v1/file/608b006d2fa25629b970f139/download'
    urlretrieve(url, TEMPLATE_MESH_FILE)

In [5]:
template_mesh = itk.meshread(TEMPLATE_MESH_FILE, itk.F)
target_mesh = itk.meshread(TARGET_MESH_FILE, itk.F)

## Compare images with ITKWidgets

We can use `view`, `compare`, and `checkerboard` to inspect mesh and image data.

In [6]:
#view(geometries=[template_mesh,target_mesh])

## Mesh-To-Image Conversion in the Base Class
Python registration classes inheriting from `MeshToMeshRegistrar` implement unique registration algorithms. The base class includes common definitions for 3D mesh-to-mesh registration, abstract methods, and a mesh-to-image conversion method.

The `mesh_to_image` function takes in a 3D mesh and converts it into an ITK 3D image object. The spacing, origin, and size of the 3D image may be calculated from the minimum bounding box of the mesh or set from a reference image.

If a list of meshes is passed to `mesh_to_image` then the output images will each be set with the same size, spacing, and origin so that every mesh is fully contained within the image region.

In [7]:
from src.hasi.hasi.meshtomeshregistrar import MeshToMeshRegistrar
registrar = MeshToMeshRegistrar()

In [8]:
target_image, template_image = registrar.mesh_to_image([target_mesh, template_mesh])

Comparing the meshes generated from the given meshes, we see that the two bone images are generally similar but do not exactly line up together. Registration will translate one mesh so that the two bones better coincide.

In [9]:
#compare(template_image,target_image)

In [10]:
#checkerboard(template_image, target_image, pattern=PATTERN_COUNT)

## Point Set Downsizing in the Base Class

The base `MeshToMeshRegistrar` class also provides decimation functionality for uniform random point set sampling from a given mesh, primarily used to improve performance for point set based registration of dense meshes. Note that uniform random sampling may not be suitable for all shapes, in which case application-specific resampling should be carried out externally prior to registration.

For subsequent registration in this notebook we will rely on a template atlas with approximately 4,000 points which does not require decimation.

In [11]:
target_points_reduced = \
    registrar.randomly_sample_mesh_points(mesh=target_mesh, sampling_rate=0.01)

In [12]:
#view(geometries=[target_mesh],point_sets=[target_points_reduced])

## Run Mean Squares Image Registration

The `MeanSquaresRegistrar` class converts meshes to images and runs [Broyden-Fletcher-Goldfarb-Shanno Optimization](https://itk.org/Doxygen/html/classitk_1_1LBFGSBOptimizerv4.html) on a [BSplineTransform](https://itk.org/Doxygen/html/classitk_1_1BSplineTransform.html) to iteratively reduce the mean square error. The resulting transform is then applied to resample the target mesh into the template mesh domain.

Progress is shown with an itkwidgets display via hooks into the ITK event-observer system. The resultant mesh is returned as an object in the Python environment and may be optionally written out to a file. Iteration updates may also be printed to the output window with the optional `verbose` flag.

In [13]:
from src.hasi.hasi.meansquaresregistrar import MeanSquaresRegistrar

In [14]:
# Must instantiate a registration object to initialize optimizers
registrar = MeanSquaresRegistrar()

In [15]:
progress = FloatProgress(
        min=0.0,
        max=150.0,
        step=1
    )
box = HBox([
    Label('Register images'),
    progress
])
box

In [16]:
def update_progress():
    progress.value = registrar.optimizer.GetCurrentIteration()
registrar.optimizer.AddObserver(itk.IterationEvent(), update_progress)

0

In [17]:
(transform_result, mesh_result) = registrar.register(template_mesh,
                                             target_mesh,
                                             filepath=MEANSQUARES_OUTPUT_FILE,
                                             verbose=True,
                                             num_iterations=200)

0 0.4953344442828443 0.0
0 0.3679928041625301 0.0
0 0.3679928041625301 0.0
1 0.1382597111142386 0.002047170989457994
1 0.1382597111142386 0.002047170989457994
2 0.09620259423214085 0.001261876484024073
2 0.09620259423214085 0.001261876484024073
3 0.0810644365222312 0.0011402423999786182
3 0.0810644365222312 0.0011402423999786182
4 0.05506265821062781 0.0008026746053761812
4 0.05506265821062781 0.0008026746053761812
5 0.036229100368238225 0.0005999657307756083
5 0.036229100368238225 0.0005999657307756083
6 0.02201122948495042 0.0006569240263049952
6 0.02201122948495042 0.0006569240263049952
7 0.019790912974060545 0.000227595005185355
7 0.019790912974060545 0.000227595005185355
8 0.017138213201441897 0.00012848767601378967
8 0.017138213201441897 0.00012848767601378967
9 0.01643926158089219 0.00011380109594120822
9 0.01643926158089219 0.00011380109594120822
10 0.015742424102559768 0.000229305332514829
10 0.015742424102559768 0.000229305332514829
11 0.015332139820951112 9.86042590450415e-0

100 0.007166497278198204 3.225428203746481e-05
100 0.007166497278198204 3.225428203746481e-05
101 0.007149650939008197 1.2455706971964613e-05
101 0.007149650939008197 1.2455706971964613e-05
102 0.007121663744128236 1.2982402927792889e-05
102 0.007121663744128236 1.2982402927792889e-05
103 0.007107483587565755 1.609346523245832e-05
103 0.007107483587565755 1.609346523245832e-05
104 0.007092042896733735 2.3113347269329504e-05
104 0.007092042896733735 2.3113347269329504e-05
105 0.0070876421136930245 3.544249173464179e-05
105 0.0070876421136930245 3.544249173464179e-05
106 0.0070856698373387254 1.011908203672434e-05
106 0.0070856698373387254 1.011908203672434e-05
107 0.007076003884998143 9.5015908970922e-06
107 0.007076003884998143 9.5015908970922e-06
108 0.0070623753300841675 1.2760766981931491e-05
108 0.0070623753300841675 1.2760766981931491e-05
109 0.007066499924291111 1.4079264563422209e-05
109 0.007056548175037092 1.4079264563422209e-05
109 0.007056548175037092 1.4079264563422209e-05


Comparison of the resulting mesh with the target shows successful registration.

In [18]:
#view(geometries=[mesh_result,target_mesh])

## Diffeomorphic Registration

The `DiffeoRegistrar` class converts meshes to images and performs registration using the [Diffeomorphic Demons registration algorithm](https://itk.org/Doxygen/html/classitk_1_1DiffeomorphicDemonsRegistrationFilter.html). The resultant deformation field is then applied to resample the target mesh into the template mesh domain.

Custom observers may print out iteration data accessed via the `registrar.filter` object.

In [19]:
from src.hasi.hasi.diffeoregistrar import DiffeoRegistrar

In [20]:
registrar = DiffeoRegistrar()

In [21]:
diffeoProgress = FloatProgress(
        min=0.0,
        max=200.0,
        step=1
    )
diffeoBox = HBox([
    Label('Register images'),
    diffeoProgress
])
diffeoBox

In [22]:
def update_diff_progress():
    diffeoProgress.value = registrar.filter.GetElapsedIterations()

registrar.filter.AddObserver(itk.IterationEvent(),update_diff_progress)

(transform_result, mesh_result) = registrar.register(template_mesh,
                                        target_mesh,
                                        filepath=DIFFEO_OUTPUT_FILE,
                                        verbose=True)

1.7976931348623157e+308
0.4953344445027531
0.4919978146151957
0.49014250375875607
0.4883152615673534
0.4876264321046578
0.4863758875349746
0.4848329850901018
0.4832380374425963
0.4816613820019586
0.47992311701764906
0.47819615858812126
0.4764180158719083
0.4746375325575763
0.4725753735317665
0.47054456107198983
0.4683531446736118
0.46619317072217226
0.4638398448774739
0.4617263143098048
0.45963088709668165
0.45765992990626214
0.45564904932109535
0.45405493940994296
0.4526567041931353
0.45126435618474847
0.4499785970781415
0.4486827830549383
0.4475157724666353
0.44628145441325595
0.4450578744943134
0.4438195343732895
0.4425877546195735
0.441339593969622
0.4401098214527867
0.4388818595160368
0.43749092474381784
0.43564875311473567
0.43370194230716685
0.4311092442088367
0.4285241607433405
0.42582081376755554
0.4231977163312322
0.4205323448942155
0.4176938430560469
0.41429447935831476
0.4105150579147669
0.4078069432083754
0.4054533839896938
0.4031936651465812
0.4008872988176601
0.398616092

Compare the translated image to the target image.

In [23]:
#view(geometries=[mesh_result, target_mesh])

# Entropy-based Registration

The `PointSetEntropyRegistrar` class registers two 3D meshes by computing the transform which minimizes entropy measures between the two point clouds. In this example we substitute a [`EuclideanDistancePointSetToPointSetMetric`](https://itk.org/Doxygen/html/classitk_1_1EuclideanDistancePointSetToPointSetMetricv4.html) to compare the two clouds.

In [8]:
from src.hasi.hasi.pointsetentropyregistrar import PointSetEntropyRegistrar
registrar = PointSetEntropyRegistrar()

In [8]:
progress = FloatProgress(
        min=0.0,
        max=2000.0,
        step=1
    )
progressBox = HBox([
    Label('Register images'),
    progress
])
progressBox

In [9]:
def update_progress():
    progressBox.value = registrar.optimizer.GetCurrentIteration()

registrar.optimizer.AddObserver(itk.IterationEvent(),update_progress)

0

In [9]:
import glob

mesh_files = glob.glob('Input\\meshes\\*.vtk')
mesh_files.sort()

In [10]:
def set_color_point_data(target):
    target.GetPointData().Reserve(target.GetNumberOfPoints())
    data = itk.array_view_from_vector_container(target.GetPointData())
    for i in range(0,data.shape[0]):
        data[i] = i

In [11]:
def set_euclidean_distance_point_data(template,target):
    assert(template.GetNumberOfPoints() == target.GetNumberOfPoints())
    
    target.GetPointData().Reserve(target.GetNumberOfPoints())
    data = itk.array_view_from_vector_container(target.GetPointData())
    
    for idx in range(target.GetNumberOfPoints()):
        data[idx] = (sum([(target.GetPoint(idx)[i] - template.GetPoint(idx)[i]) ** 2 for i in range(3)])) ** 0.5

In [27]:
mesh_results = list()
for idx in range(len(mesh_files)):
    target_mesh = itk.meshread(mesh_files[idx], itk.F)
    metric = itk.EuclideanDistancePointSetToPointSetMetricv4[itk.PointSet[itk.F,3]].New()

    registrar = PointSetEntropyRegistrar()
    (transform_result, mesh_result) = registrar.register(
                           template_mesh=template_mesh,
                           target_mesh=target_mesh,
                           metric=metric,
                           #filepath=POINTSET_OUTPUT_FILE,
                           verbose=True,
                           learning_rate=1.0,
                           max_iterations=300,
                           resample_from_target=True)
    mesh_results.append(mesh_result)

0 0.0
0 0.20032173540867496
1 0.14864390134543362
2 0.12320818891097145
3 0.10802017775965848
4 0.09715986232703265
5 0.08916955976677314
6 0.08312357678962795
7 0.07849107790268896
8 0.07481486435903528
9 0.07193538382530532
10 0.06966422375066066
11 0.06782247607513572
12 0.0662364561897577
13 0.06491432361908861
14 0.06377511384895661
15 0.06281752600361236
16 0.06198324757298054
17 0.06122087017244233
18 0.06053736253791672
19 0.05991704652854803
20 0.05934399634233393
21 0.05881111962029392
22 0.058330457397190144
23 0.05788496203223169
24 0.057465084801870694
25 0.05707270637604952
26 0.05670012139568885
27 0.05633767115237087
28 0.05598707479475316
29 0.0556413791894961
30 0.05530028181161485
31 0.05497844313488113
32 0.05465657959889966
33 0.05435381233739907
34 0.05406140936344617
35 0.053777908268194986
36 0.05351293796638151
37 0.05325923455391663
38 0.053007459063426625
39 0.052757111808189074
40 0.052511433192933485
41 0.052260786849444095
42 0.05201423561745963
43 0.05178

20 0.05652837680168472
21 0.05530507143716108
22 0.054134602834920165
23 0.053020636495240706
24 0.051944194750049855
25 0.05091371680519884
26 0.04990854226515862
27 0.048950925455557744
28 0.048024307153977146
29 0.04712948446720358
30 0.0462846441709203
31 0.04545128218920508
32 0.044643358857768024
33 0.043888957619121746
34 0.04316445250215185
35 0.04248362994983003
36 0.04183262747532614
37 0.0412037429531749
38 0.04059294224542183
39 0.040016154126255296
40 0.03945396290133864
41 0.03892002005470285
42 0.03840443107046075
43 0.03791056816607884
44 0.037459777174137766
45 0.03701969721014922
46 0.03658629120129073
47 0.036148671462410145
48 0.03571479106454752
49 0.035295001371293634
50 0.034907069624876136
51 0.03453613144565263
52 0.03417757083123989
53 0.03382311711899252
54 0.03348168950333608
55 0.03316049376927745
56 0.032854504731590364
57 0.03257361869598419
58 0.0323092692810875
59 0.03205047461546761
60 0.03179900730212333
61 0.03154807758579675
62 0.03128955525259092
6

37 0.06203331251790044
38 0.06200254660223383
39 0.061975489496719285
40 0.06194825397931028
41 0.06191284909824855
42 0.06187439307934048
43 0.06183781175635154
44 0.06178344717908907
45 0.06173854980664441
46 0.06170285531315976
47 0.061666775700963794
48 0.06163621660668886
49 0.06160604360916199
50 0.06157953673992406
51 0.061546883377460275
52 0.06152272818300061
53 0.061496361026742795
54 0.06147458078975856
55 0.061453645952823904
56 0.06143348986737493
57 0.061411765662779935
58 0.06138959480217494
59 0.06136590479985754
60 0.0613449821629016
61 0.06132511471042944
62 0.06130362715176032
63 0.06128941722610892
64 0.06128030170601413
65 0.06126911991521983
66 0.06126156471034367
67 0.061252549914251754
68 0.06124381357526593
69 0.061232094320861595
70 0.061219718757816316
71 0.06120838964874548
72 0.061198097806139815
73 0.06116753294943587
74 0.061146808156366736
75 0.06112898079862582
76 0.06111015401299866
77 0.06109116498124756
78 0.06107563803554773
79 0.06106195127534013
8

60 0.03719478179831917
61 0.036981018254859985
62 0.03675685609813149
63 0.03654743840142188
64 0.03634450549369557
65 0.03615265732726049
66 0.035960615339934346
67 0.035777871035599894
68 0.03560241198290828
69 0.03544359380475708
70 0.03528889692627067
71 0.03512863046215038
72 0.034976273855205714
73 0.034821220070147804
74 0.0346613299727337
75 0.034515582202940556
76 0.034379093643732185
77 0.03425717670762239
78 0.03414281837394924
79 0.034036573574504285
80 0.03393030872178001
81 0.03382315883586527
82 0.033718426218087176
83 0.03362330203997862
84 0.03352835091341494
85 0.033431142159209344
86 0.03334499247192542
87 0.03325969894232424
88 0.033181172606110525
89 0.03310602139806973
90 0.03303276574122699
91 0.03296638825145232
92 0.03289891447467815
93 0.032832860115590244
94 0.03276975272164454
95 0.032709882607610014
96 0.032652171663020896
97 0.03259768762233917
98 0.03253977057040234
99 0.03248170071202307
100 0.03243063474451154
101 0.03238209585901326
102 0.0323339464355

74 0.053907741393838866
75 0.053816038819235856
76 0.05371815906534015
77 0.05362223172898587
78 0.05352735643586035
79 0.053437718258477214
80 0.053352329653964324
81 0.05326767211043322
82 0.053179581679579736
83 0.053093144295146735
84 0.05300895607685094
85 0.0529277945860366
86 0.052846778699354974
87 0.05276548459419518
88 0.05268573035533261
89 0.052610844966873016
90 0.05254205456405691
91 0.05247588231440295
92 0.05240825059900456
93 0.05233363226399049
94 0.0522566781413545
95 0.052183143261049184
96 0.052111043945087206
97 0.05204127368078755
98 0.05197920804032017
99 0.05191775411751292
100 0.051859752386200574
101 0.051802714183650046
102 0.05174215054670083
103 0.05168353044020937
104 0.05162349262696961
105 0.05156428651795449
106 0.05150702744415294
107 0.05144923634310058
108 0.05139040602950213
109 0.05133134057320995
110 0.051275945759079976
111 0.05122345178625777
112 0.051169214272747725
113 0.051110126993441696
114 0.051055527070544116
115 0.05100169204599631
116 

...

184 0.057626738773081436
185 0.05761254661869522
186 0.05759869643616573
187 0.0575854296655437
188 0.057568072721060216
189 0.05755483321478074
190 0.05754562743968061
191 0.05753467908042778
192 0.057526868293351206
193 0.05751877125663751
194 0.05751140113550169
195 0.057502698993882884
196 0.05749473028606647
197 0.057485279082811
198 0.057474218815681544
199 0.05746205538857943
200 0.05745093259086786
201 0.057439193146400246
202 0.05743203071166553
203 0.0574250826825182
204 0.05741737604999873
205 0.057406032294318426
206 0.05739204587719876
207 0.05737954534187469
208 0.05736800499751167
209 0.0573582719692254
210 0.05735028217152734
211 0.057341382437058434
212 0.05733545651757997
213 0.05732969680784519
214 0.05732435682709872
215 0.05731870417098844
216 0.057315456606289186
217 0.05731066340279194
218 0.05730236352052414
219 0.057294745001787765
220 0.05728867793041928
221 0.057275739669635624
222 0.05726546358725515
223 0.057256508753514984
224 0.05724818857397006
225 0.057

218 0.04342048693192246
219 0.043417627703575205
220 0.04341418504926815
221 0.043413572815428764
222 0.043413897454454536
223 0.04341475217849048
224 0.0434135470939854
225 0.04341398269919989
226 0.043413276865074034
227 0.04341103998956521
228 0.04340926202185366
229 0.04340314930327577
230 0.04339930918477145
231 0.043391033352714846
232 0.04338509907356543
233 0.04338366950757602
234 0.043381493891503566
235 0.043380977318038506
236 0.0433790392151606
237 0.043380565814595505
238 0.04337893397655046
239 0.043377257993474916
240 0.043375418131075226
241 0.04337462104654914
242 0.043370072132740495
243 0.043364748447363516
244 0.04335763289498389
245 0.04335221835797445
246 0.043344392609385565
247 0.043338421892328324
248 0.04333250529842578
249 0.043328475772037654
250 0.0433245734868079
251 0.04332325828012593
252 0.043324857496481024
253 0.043325713306784704
254 0.043327920480415574
255 0.04332912642815893
256 0.04333004036848723
257 0.04332861134033812
258 0.04332906130296478
2

239 0.05578705925327405
240 0.05578838249550432
241 0.055788976003978835
242 0.055790654820356306
243 0.05579201620255074
244 0.05579396860877655
245 0.05579516811217223
246 0.05579860922120171
247 0.055801463503716196
248 0.0558042318366586
249 0.055806974522505
250 0.05580913559171659
251 0.05581127081059499
252 0.0558133182859725
253 0.05581490372073914
254 0.05581595177301505
255 0.05581685021694029
256 0.0558177239533851
257 0.05581892142143984
258 0.05581961134143456
259 0.05582033415209609
260 0.05582061261967654
261 0.05582225647844759
262 0.055824445150258625
263 0.055826919803842905
264 0.055830634759218435
265 0.05583403725016738
266 0.055838055057948026
267 0.055841553531809106
268 0.05584428769006832
269 0.05584690240367515
270 0.055849757188735395
271 0.055852578381033836
272 0.05585464535732476
273 0.055856546482494544
274 0.05585883649733618
275 0.05586202843478791
276 0.05586499977670551
277 0.05586767451621734
278 0.05587040752799291
279 0.05587302870840187
280 0.0558

263 0.036258178201274915
264 0.03624555211663159
265 0.03623271031831358
266 0.036220265127658545
267 0.036208720506932454
268 0.0361966328350284
269 0.036183686682097406
270 0.03617189922547935
271 0.03616008744092298
272 0.03614998761370799
273 0.03614016222797566
274 0.036130120474765604
275 0.036121569087148316
276 0.03611296193936648
277 0.03610424866481215
278 0.03609711051108929
279 0.03608932107149991
280 0.03608059439506133
281 0.036071608951557664
282 0.03606316334836991
283 0.036056297819427256
284 0.03604663380927438
285 0.03603955287383814
286 0.03603013973837905
287 0.036021941105400454
288 0.0360145034214074
289 0.036007814424298436
290 0.03599989004430802
291 0.03599184828940766
292 0.035983188694372115
293 0.03597678107772049
294 0.03597191313603904
295 0.03596501915932582
296 0.03595909890931065
297 0.03595396301755728
298 0.03594624837121488
299 0.035937570905684245
300 0.035937570905684245
Number of iterations run: 300
Final metric value: 0.035937570905684245
Final 

292 0.03769359627697946
293 0.037688211739764874
294 0.03768360563077231
295 0.03767932105163531
296 0.037674220194527305
297 0.03766893237196451
298 0.03766409872633147
299 0.03765874062716655
300 0.03765874062716655
Number of iterations run: 300
Final metric value: 0.03765874062716655
Final transform position: [0.9958096951516051, -0.09275793734014512, 0.13188123162081697, 0.1097927491919694, 0.9873374912917787, -0.21568698751317122, -0.0970431903110491, 0.2426889777801398, 0.9882549475559143, -0.09247684019336712, -0.48905074484239636, -0.4395013881302035]
Optimizer scales: [11.744144658808226, 4.475390574611993, 2.5780283025156336, 11.744144658808226, 4.475390574611993, 2.5780283025156336, 11.744144658808226, 4.475390574611993, 2.5780283025156336, 1.0000000000000018, 1.0000000000000018, 1.0000000000000018]
Optimizer learning rate: 1.0
0 0.0
0 0.30820461108999325
1 0.18603940109348405
2 0.14473580009018272
3 0.12806296387324495
4 0.12090410712942996
5 0.11705899654655172
6 0.1144087

In [28]:
# TODO procrustes alignment after resampling for accurate distances
align_filter = itk.MeshProcrustesAlignFilter[type(template_mesh),type(template_mesh)].New()

align_filter.SetUseInitialAverageOff()
align_filter.SetUseNormalizationOff()
align_filter.SetUseScalingOff()
align_filter.SetConvergence(8e-2)

align_filter.SetNumberOfInputs(len(mesh_results))
for idx in range(len(mesh_results)):
    align_filter.SetInput(idx, mesh_results[idx])
    
align_filter.Update()

align_results = list()
for idx in range(len(mesh_results)):
    align_results.append(align_filter.GetOutput(idx))
    #set_color_point_data(align_results[idx])
    set_euclidean_distance_point_data(template_mesh, align_results[idx])


In [29]:
for idx in range(len(align_results)):
    itk.meshwrite(align_results[idx], f'Output\\correspondence\\{mesh_files[idx][13:-4]}.vtk')

In [28]:
#view(geometries=[mesh_result,target_mesh])

## Resample From Target

A common procedure for comparing correspondences across samples is to register an atlas to each mesh sample and then deform the template to align with points on the target surface. The `MeshToMeshRegistrar` class provides an interface to use ITK's [`KdTree`](https://itk.org/Doxygen/html/classitk_1_1Statistics_1_1KdTree.html) to set each template point to its nearest neighbor on the target mesh.

In [11]:
template_resampled = \
    registrar.resample_template_from_target(mesh_result, target_mesh)

itk.meshwrite(template_resampled,POINTSET_RESAMPLED_OUTPUT_FILE)

In [ ]:
#view(geometries=[target_mesh,mesh_result,template_resampled])

In [ ]:
# Clean up file output
os.remove(MEANSQUARES_OUTPUT_FILE)
os.remove(DIFFEO_OUTPUT_FILE)
os.remove(POINTSET_OUTPUT_FILE)
os.remove(POINTSET_RESAMPLED_OUTPUT_FILE)